<a href="https://colab.research.google.com/github/Naeima/image/blob/main/imageMapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install rdflib for RDF handling
!pip install rdflib

from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, SKOS, OWL
import pandas as pd
from google.colab import files

# Upload CSV file
uploaded = files.upload()

# Define namespaces
FOO = Namespace("https://w3id.org/def/foo#")

# Create an RDF graph
g = Graph()

# Bind namespaces to prefixes
g.bind("foo", FOO)
g.bind("rdfs", RDFS)
g.bind("skos", SKOS)
g.bind("owl", OWL)

# Camera Trap as a Sensor
camera_trap_uri = URIRef(FOO.CameraTrap)
g.add((camera_trap_uri, RDF.type, FOO.Sensor))
g.add((camera_trap_uri, RDFS.label, Literal("Camera Trap", lang="en")))
g.add((camera_trap_uri, FOO.observes, FOO.imageObservation))
g.add((camera_trap_uri, FOO.model, Literal("Model", lang="en")))
g.add((camera_trap_uri, FOO.make, Literal("Make", lang="en")))
g.add((camera_trap_uri, FOO.hasFeatureOfInterest, FOO.Image))

# Image as a Class
image_class_uri = URIRef(FOO.Image)
g.add((image_class_uri, RDF.type, OWL.Class))
g.add((image_class_uri, RDFS.label, Literal("Camera Trap Image", lang="en")))
g.add((image_class_uri, SKOS.definition, Literal("Image generated by motion-activated wildlife cameras.", lang="en")))
g.add((image_class_uri, RDFS.definedBy, URIRef("http://w3id.org/def/foo#")))

# Process the uploaded CSV file and create RDF mappings
for filename in uploaded.keys():
    df = pd.read_csv(filename)
    for index, row in df.iterrows():
        # Extract fields (adjust these to match your CSV columns)
        image_id = row['Image_ID']  # Example column name in your CSV
        timestamp = row['Timestamp']
        location = row['Location']
        species = row['Species']
        image_file = row['Image_File']  # Example column for the image filename

        # Image Observation
        observation_uri = URIRef(f"https://w3id.org/def/foo#imageObservation_{image_id}")
        g.add((observation_uri, RDF.type, FOO.Observation))
        g.add((observation_uri, RDFS.label, Literal("Image Observation", lang="en")))
        g.add((observation_uri, FOO.hasFeatureOfInterest, FOO.Image))
        g.add((observation_uri, FOO.madeBySensor, FOO.CameraTrap))
        g.add((observation_uri, FOO.imageFile, Literal(image_file, datatype=XSD.string)))
        g.add((observation_uri, FOO.cameraLocation, Literal(location, datatype=XSD.string)))
        g.add((observation_uri, FOO.animalDetected, Literal(species, datatype=XSD.string)))
        g.add((observation_uri, FOO.timestamp, Literal(timestamp, datatype=XSD.dateTime)))

# Serialize the RDF graph to Turtle format
output_file = "camera_trap_image_observations.ttl"
g.serialize(destination=output_file, format="turtle")

# Download the resulting RDF Turtle file
files.download(output_file)

print(f"RDF data has been serialized to {output_file}")